In [ ]:
import os
import sys
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

fontsize = 8
lw = 0.75

matplotlib.rc('font', **{'family': 'Times New Roman', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

%matplotlib inline

if '..' not in sys.path:
    sys.path.append('..')
from dlml.utils import collect_experiments

In [ ]:
def make_axes(rows, cols, x_offset, y_offset, x_space, y_space, squeeze=True):
    w = (1 - np.sum(x_offset) - x_space * (cols - 1)) / cols
    h = (1 - np.sum(y_offset) - y_space * (rows - 1)) / rows
    
    ax = [[plt.axes([x_offset[0] + (w + x_space) * j,
                     y_offset[0] + (h + y_space) * i,
                     w, h]) for j in range(cols)] for i in range(rows-1, -1, -1)]
    
    for row in ax:
        for a in row:
            for side in 'right','top':
                a.spines[side].set_visible(False)

    if squeeze:
        if rows == 1 and cols == 1:
            return ax[0][0]
        if rows == 1:
            return ax[0]
        if cols == 1:
            return [a[0] for a in ax]
        
    return ax

### Figure for the PSCC-2022 abstract

### Figure 1 for the PSCC-2022 paper

#### Find the best experiment that matches a given set of tags

In [ ]:
area_ID = 1
area_measure = 'momentum'
D = 2
DZA = 0
outfile = f'experiments_area_{area_ID}_D_{D:d}_DZA_{DZA:g}_{area_measure}.pkl'
force = False
if not os.path.isfile(outfile) or force:
    experiments = collect_experiments(area_ID, area_measure=area_measure, D=2, DZA=0, verbose=True)
    pickle.dump(experiments, open(outfile,  'wb'))
else:
    experiments = pickle.load(open(outfile, 'rb'))
    print(f'Loaded experiments data from {outfile}.')

data = {}
for ID,expt in experiments.items():
    val_loss = expt['val_loss'].min()
    loss = expt['loss'].min()
    MAPE = expt['MAPE']
    for tag in expt['tags']:
        if tag.startswith('buses'):
            buses = tag.split('_')[1]
            break
    try:
        data[buses]['IDs'].append(ID)
        data[buses]['val_loss'].append(val_loss)
        data[buses]['loss'].append(loss)
        data[buses]['MAPE'].append(MAPE)
        data[buses]['with_ReLU'].append('ReLU_none' not in expt['tags'])
    except:
        data[buses] = {
            'IDs': [ID],
            'val_loss': [val_loss],
            'loss': [loss],
            'MAPE': [MAPE],
            'with_ReLU': ['ReLU_none' not in expt['tags']]
        }
for key in data:
    data[key]['val_loss'] = np.array(data[key]['val_loss'])
    data[key]['loss'] = np.array(data[key]['loss'])
    data[key]['MAPE'] = np.array(data[key]['MAPE'])
    data[key]['with_ReLU'] = np.array(data[key]['with_ReLU'])

experiment_IDs = list(experiments.keys())
experiment_ID = experiment_IDs[np.argmin([expt['val_loss'].min() for expt in experiments.values()])]
MAPE = experiments[experiment_ID]['MAPE']
loss = experiments[experiment_ID]['loss']
val_loss = experiments[experiment_ID]['val_loss']
batch_loss = experiments[experiment_ID]['batch_loss']
tags = experiments[experiment_ID]['tags']
print(f'The best experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE:.4f}%).')

#### Load the experiment parameters and the test results

In [ ]:
experiments_path = '../experiments/neural_network/'
network_parameters = pickle.load(open(experiments_path + experiment_ID + '/parameters.pkl', 'rb'))
test_results = pickle.load(open(experiments_path + experiment_ID + '/test_results.pkl', 'rb'))

In [ ]:
first_letter = lambda s: s[0]
expts = []
data_filtered = {}
for buses in data:
    data_filtered[buses] = {'IDs': [], 'val_loss': [], 'loss': [], 'MAPE': [], 'with_ReLU': []}
    for i,expt in enumerate(data[buses]['IDs']):
        pars = pickle.load(open(f'../experiments/neural_network/{expt}/parameters.pkl', 'rb'))
        var_names = pars['var_names']
        if 'Q' in list(map(first_letter, var_names)):
            expts.append(expt)
            data_filtered[buses]['IDs'].append(expt)
            data_filtered[buses]['val_loss'].append(data[buses]['val_loss'][i])
            data_filtered[buses]['loss'].append(data[buses]['loss'][i])
            data_filtered[buses]['MAPE'].append(data[buses]['MAPE'][i])
            data_filtered[buses]['with_ReLU'].append(data[buses]['with_ReLU'][i])
    data_filtered[buses]['val_loss'] = np.array(data_filtered[buses]['val_loss'])
    data_filtered[buses]['loss'] = np.array(data_filtered[buses]['loss'])
    data_filtered[buses]['MAPE'] = np.array(data_filtered[buses]['MAPE'])
    data_filtered[buses]['with_ReLU'] = np.array(data_filtered[buses]['with_ReLU'])

In [ ]:
# # fig,ax = plt.subplots(1, 3, figsize=(5, 1.8))
# fig,ax = plt.subplots(3, 1, figsize=(2.5, 4.5))

rows = 3
cols = 1
x_offset = [0.2, 0.12]
y_offset = [0.1, 0.03]
x_space = 0.1
y_space = 0.14
fig = plt.figure(figsize=(7/2.54, 10/2.54))
ax = make_axes(rows, cols, x_offset, y_offset, x_space, y_space)

keys = ['3', '14', '17', '3-14', '3-17', '14-17', '3-14-17']
n_keys = len(keys)
ylims_left = {'inertia': [0.18, 0.82], 'energy': [0.045, 0.26], 'momentum': [0.002, 0.013]}
ylims_right = {'inertia': [0.47, 2.2], 'energy': [0.47, 2.2], 'momentum': [0.7, 3.5]}
ylim_left = ylims_left[area_measure]
ylim_right = ylims_right[area_measure]
dy_left = np.diff(ylim_left)
dy_right = np.diff(ylim_right)
ax = np.append(ax, ax[0].twinx())
offset = 0.3
for i,key in enumerate(keys):
    if key in data_filtered:
        # without ReLU
        idx, = np.where(~data_filtered[key]['with_ReLU'])
        for j in idx:
            yl = data_filtered[key]['val_loss'][j]
            yr = ylim_left[0] + ((data_filtered[key]['MAPE'][j] - ylim_right[0]) / dy_right) * dy_left
            ax[0].plot([i + 1 - offset, i + 1 + offset], [yl, yr], '-', color=[.4,.4,.4], lw=0.75)
        ax[0].plot(i + 1 - offset + np.zeros(idx.size), data_filtered[key]['val_loss'][idx], 'ko',
                   markerfacecolor='w', markersize=3, markeredgewidth=1)
        ax[-1].plot(i + 1 + offset + np.zeros(idx.size), data_filtered[key]['MAPE'][idx], 'ro',
                    markerfacecolor='w', markersize=3, markeredgewidth=1)

        # with ReLU
        idx, = np.where(data_filtered[key]['with_ReLU'])
        for j in idx:
            yl = data_filtered[key]['val_loss'][j]
            yr = ylim_left[0] + ((data_filtered[key]['MAPE'][j] - ylim_right[0]) / dy_right) * dy_left
            ax[0].plot([i + 1 - offset, i + 1 + offset], [yl, yr], '-', color=[.4,.4,.4], lw=0.75)
        ax[0].plot(i + 1 - offset + np.zeros(idx.size), data_filtered[key]['val_loss'][idx], 'ks',
                   markerfacecolor='k', markersize=2.5, markeredgewidth=1)
        ax[-1].plot(i + 1 + offset + np.zeros(idx.size), data_filtered[key]['MAPE'][idx], 'rs',
                   markerfacecolor='r', markersize=2.5, markeredgewidth=1)
ax[-1].tick_params(axis='y', labelcolor='r')
ax[0].set_xlim([1 - offset * 1.75, n_keys + offset * 1.75])
ax[0].set_xticks(1 + np.arange(n_keys))
ax[0].set_xticklabels(keys, rotation=45)
ax[0].set_xlabel('Buses')
ax[0].set_ylabel('Min. validation loss')
ax[-1].set_ylabel('MAPE [%]', color='r')
ax[0].grid(which='major', axis='x', linestyle=':', linewidth=0.5, color=[.6,.6,.6])
ax[0].set_ylim(ylim_left)
ax[-1].set_ylim(ylim_right)

n_epochs = len(loss)
epochs = np.arange(n_epochs) + 1
ax[1].semilogy(epochs, val_loss, color=[.8,.8,.8], lw=1, label='Validation set')
ax[1].semilogy(epochs, loss, 'k', lw=1, label='Training set')
ax[1].legend(loc='upper right', fontsize=7, frameon=False)
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('MAE')
xticks = {'inertia': np.r_[0 : 800 : 150], 'energy': np.r_[0 : 700 : 150], 'momentum': np.r_[0 : 1050 : 200]}
ax[1].set_xticks(xticks[area_measure])
y_test, y_prediction = test_results['y_test'], test_results['y_prediction']
line_pos = {'inertia': [26, 48.5], 'energy': [8, 14.5], 'momentum': [0.26, 0.49]}
ax[2].plot(line_pos[area_measure], line_pos[area_measure], lw=2, color=[.8,.8,.8])
ax[2].plot(y_test, y_prediction, 'o', color=[.6,.6,.6], markerfacecolor='w', markersize=3, markeredgewidth=0.5)
for x in np.unique(y_test):
    idx, = np.where(y_test == x)
    ymean, ystd = y_prediction[idx].mean(), y_prediction[idx].std()
    ysem = ystd / np.sqrt(len(idx))
    ax[2].plot(x + np.zeros(2), ymean + 3 * ystd * np.array([-1,1]), 'k', linewidth=1)
    ax[2].plot(x, ymean, 'ko', markerfacecolor='w', markersize=3.75, markeredgewidth=1)
units = {'inertia': 's', 'energy': 'GW$\cdot$s', 'momentum': 'GW$\cdot$s$^2$'}
ax[2].set_xlabel(r'{} [{}]'.format(area_measure.capitalize(), units[area_measure]))
ax[2].set_ylabel(r'Predicted {} [{}]'.format(area_measure, units[area_measure]))
ticks = {'inertia': np.r_[26:51:4], 'energy': np.r_[8:16], 'momentum': np.r_[0.2:0.55:0.05]}
ax[2].set_xticks(ticks[area_measure])
ax[2].set_yticks(ticks[area_measure])
limits = {'inertia': [24.5,50], 'energy': [7.5,15], 'momentum': [0.25,0.51]}
ax[2].set_xlim(limits[area_measure])
ax[2].set_ylim(limits[area_measure])
text_pos = {'inertia': [27, 46], 'energy': [8.5, 14.5], 'momentum': [0.3, 0.5]}
text_pos = {key: [limits[key][0] + 0.05 * np.diff(limits[key]), limits[key][1] - 0.175 * np.diff(limits[key])] \
           for key in limits}
ax[2].text(text_pos[area_measure][0], text_pos[area_measure][1], f'MAPE = {MAPE:.2f}%', fontsize=7)
ax[2].grid(which='major', axis='y', lw=0.5, ls=':')

for a in ax[1:3]:
    for side in 'top','right':
        a.spines[side].set_visible(False)

# fig.tight_layout(pad=0.1)
outfile = f'training_results_area_{area_ID}_D_{D:d}_DZA_{DZA:.1f}_{area_measure}_{experiment_ID[:6]}.pdf'
fig.savefig(outfile)

### Figure with training results comparison

IEEE39 network, inertia changed in area 1